<a href="https://colab.research.google.com/github/lunawatyash279/Document-Scanner/blob/master/project_code_cnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install pyedflib


In [0]:
!wget -r --no-parent https://physionet.org/pn6/chbmit/

--2019-05-30 06:44:54--  https://physionet.org/pn6/chbmit/
Resolving physionet.org (physionet.org)... 128.30.30.88
Connecting to physionet.org (physionet.org)|128.30.30.88|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/html]
Saving to: ‘physionet.org/pn6/chbmit/index.html’

physionet.org/pn6/c     [ <=>                ]  25.16K  --.-KB/s    in 0.07s   

2019-05-30 06:44:54 (336 KB/s) - ‘physionet.org/pn6/chbmit/index.html’ saved [25767]

Loading robots.txt; please ignore errors.
--2019-05-30 06:44:54--  https://physionet.org/robots.txt
Reusing existing connection to physionet.org:443.
HTTP request sent, awaiting response... 200 OK
Length: 106 [text/plain]
Saving to: ‘physionet.org/robots.txt’

physionet.org/robot 100%[===================>]     106  --.-KB/s    in 0s      

2019-05-30 06:44:54 (21.8 MB/s) - ‘physionet.org/robots.txt’ saved [106/106]

--2019-05-30 06:44:54--  https://physionet.org/pn6/chbmit/local.css
Reusing existing connecti

In [0]:
!mv /content/chb19-summary.txt /content/physionet.org/pn6/chbmit/chb19

**DataGram/Spectogram**

In [0]:
import os
from datetime import datetime, timedelta

import matplotlib.pyplot as plt
import numpy as np
import pyedflib
from scipy import signal
from scipy.signal import butter, lfilter

# DATASET: https://physionet.org/pn6/chbmit/
sampleRate = 256
pathDataSet = './physionet.org/pn6/chbmit/'  # path of the dataset
FirstPartPathOutput = './project/dataset/spectograms'  # path where the spectogram will be saved
#patients = ["01", "02", "03", "05", "09", "10", "14", "18", "19", "21", "23"]
# nSeizure = [7, 3, 6, 5, 4, 6, 5, 5, 6, 3, 5, 4, 5]
patients = ["09", "10", "14", "18"]
_30_MINUTES_OF_DATA = 256 * 60 * 30
_MINUTES_OF_DATA_BETWEEN_PRE_AND_SEIZURE = 3  # In theory 5 as the SPH but set to 3 to consider some seizure taken in the paperrese nel paper
_MINUTES_OF_PREICTAL = 30
_SIZE_WINDOW_IN_SECONDS = 30
_SIZE_WINDOW_SPECTOGRAM = _SIZE_WINDOW_IN_SECONDS * 256
nSpectogram = 0
signalsBlock = None
SecondPartPathOutput = ''
legendOfOutput = ''
isPreictal = ''


def loadParametersFromFile(filePath):
    global pathDataSet
    global FirstPartPathOutput
    if os.path.isfile(filePath):
        with open(filePath, "r") as f:
            line = f.readline()
            if line.split(":")[0] == "pathDataSet":
                pathDataSet = line.split(":")[1].strip()
            line = f.readline()
            if line.split(":")[0] == "FirstPartPathOutput":
                FirstPartPathOutput = line.split(":")[1].strip()


# Band size filter
def butter_bandstop_filter(data, lowcut, highcut, fs, order):
    nyq = 0.5 * fs
    low = lowcut / nyq
    high = highcut / nyq

    i, u = butter(order, [low, high], btype='bandstop')
    y = lfilter(i, u, data)
    return y


# Band size filter,high pass
def butter_highpass_filter(data, cutoff, fs, order=5):
    nyq = 0.5 * fs
    normal_cutoff = cutoff / nyq
    b, a = butter(order, normal_cutoff, btype='high', analog=False)
    y = lfilter(b, a, data)
    return y


# Creating the pointer to the patient file with index equal to index
def loadSummaryPatient(index):
    f = open(pathDataSet + 'chb' + patients[index] + '/chb' + patients[index] + '-summary.txt', 'r')
    return f


# Caricamento dei dati di un paziente(indexPatient). I dati sono presi dal file con il nome indicato in fileOfData
# Restituisce un vettore numpy con i dati del paziente contenuti nel file
def loadDataOfPatient(indexPatient, fileOfData):
    f = pyedflib.EdfReader(pathDataSet + 'chb' + patients[
        indexPatient] + '/' + fileOfData)  # https://pyedflib.readthedocs.io/en/latest/#description
    n = f.signals_in_file
    sigbufs = np.zeros((n, f.getNSamples()[0]))
    for i in np.arange(n):
        sigbufs[i, :] = f.readSignal(i)
    sigbufs = cleanData(sigbufs, indexPatient)
    return sigbufs


def cleanData(Data, indexPatient):
    if (patients[indexPatient] in ["19", "21"]):
        Data = np.delete(Data, 22, axis=0)
        Data = np.delete(Data, 17, axis=0)
        Data = np.delete(Data, 12, axis=0)
        Data = np.delete(Data, 9, axis=0)
        Data = np.delete(Data, 4, axis=0)
    return Data


def getTime(dateInString):
    time = 0
    try:
        time = datetime.strptime(dateInString, '%H:%M:%S')
    except ValueError:
        dateInString = " " + dateInString
        if (' 24' in dateInString):
            dateInString = dateInString.replace(' 24', '23')
            time = datetime.strptime(dateInString, '%H:%M:%S')
            time += timedelta(hours=1)
        else:
            dateInString = dateInString.replace(' 25', '23')
            time = datetime.strptime(dateInString, '%H:%M:%S')
            time += timedelta(hours=2)
    return time


def saveSignalsOnDisk(signalsBlock, nSpectogram):
    global SecondPartPathOutput
    global FirstPartPathOutput
    global legendOfOutput
    global isPreictal

    if not os.path.exists(FirstPartPathOutput):
        os.makedirs(FirstPartPathOutput)
    if not os.path.exists(FirstPartPathOutput + SecondPartPathOutput):
        os.makedirs(FirstPartPathOutput + SecondPartPathOutput)
    np.save(FirstPartPathOutput + SecondPartPathOutput + '/spec_' + isPreictal + '_' + str(
        nSpectogram - signalsBlock.shape[0]) + '_' + str(nSpectogram - 1), signalsBlock)
    legendOfOutput = legendOfOutput + str(nSpectogram - signalsBlock.shape[0]) + ' ' + str(
        nSpectogram - 1) + ' ' + SecondPartPathOutput + '/spec_' + isPreictal + '_' + str(
        nSpectogram - signalsBlock.shape[0]) + '_' + str(nSpectogram - 1) + '.npy\n'


def createSpectrogram(data, S=0):
    global nSpectogram
    global signalsBlock
    global inB
    signals = np.zeros((22, 59, 114))

    t = 0
    movement = int(S * 256)
    if (S == 0):
        movement = _SIZE_WINDOW_SPECTOGRAM
    while data.shape[1] - (t * movement + _SIZE_WINDOW_SPECTOGRAM) > 0:
        # CREAZIONE DELLO SPETROGRAMMA PER TUTTI I CANALI
        for i in range(0, 22):
            start = t * movement
            stop = start + _SIZE_WINDOW_SPECTOGRAM
            signals[i, :] = createSpec(data[i, start:stop])
        if (signalsBlock is None):
            signalsBlock = np.array([signals])
        else:
            signalsBlock = np.append(signalsBlock, [signals], axis=0)
        nSpectogram = nSpectogram + 1
        if (signalsBlock.shape[0] == 50):
            saveSignalsOnDisk(signalsBlock, nSpectogram)
            signalsBlock = None
            # SALVATAGGIO DI SIGNALS  
        t = t + 1
    return (data.shape[1] - t * _SIZE_WINDOW_SPECTOGRAM) * -1


def createSpec(data):
    fs = 256
    lowcut = 117
    highcut = 123

    y = butter_bandstop_filter(data, lowcut, highcut, fs, order=6)
    lowcut = 57
    highcut = 63
    y = butter_bandstop_filter(y, lowcut, highcut, fs, order=6)

    cutoff = 1
    y = butter_highpass_filter(y, cutoff, fs, order=6)

    Pxx = signal.spectrogram(y, nfft=256, fs=256, return_onesided=True, noverlap=128)[2]
    Pxx = np.delete(Pxx, np.s_[117:123 + 1], axis=0)
    Pxx = np.delete(Pxx, np.s_[57:63 + 1], axis=0)
    Pxx = np.delete(Pxx, 0, axis=0)

    result = (10 * np.log10(np.transpose(Pxx)) - (10 * np.log10(np.transpose(Pxx))).min()) / (
            10 * np.log10(np.transpose(Pxx))).ptp()
    return result


def createSpecAndPlot(data):
    freqs, bins, Pxx = signal.spectrogram(data, nfft=256, fs=256, return_onesided=True, noverlap=128)

    print("Original")
    plt.pcolormesh(freqs, bins, 10 * np.log10(np.transpose(Pxx)), cmap=plt.cm.jet)
    plt.colorbar()
    plt.ylabel('sec')
    plt.xlabel('Hz')
    plt.title('Spettrogramma')
    plt.show()
    plt.close()

    fs = 256
    lowcut = 117
    highcut = 123

    y = butter_bandstop_filter(data, lowcut, highcut, fs, order=6)
    lowcut = 57
    highcut = 63
    y = butter_bandstop_filter(y, lowcut, highcut, fs, order=6)

    cutoff = 1
    y = butter_highpass_filter(y, cutoff, fs, order=6)

    # Pxx=signal.spectrogram(y, nfft=256, fs=256, return_onesided=True, noverlap=128)[2]
    freqs, bins, Pxx = signal.spectrogram(y, nfft=256, fs=256, return_onesided=True, noverlap=128)

    print("Filtered")
    plt.pcolormesh(freqs, bins, 10 * np.log10(np.transpose(Pxx)), cmap=plt.cm.jet)
    plt.colorbar()
    plt.ylabel('sec')
    plt.xlabel('Hz')
    plt.title('Spettrogramma')
    plt.show()
    plt.close()

    Pxx = np.delete(Pxx, np.s_[117:123 + 1], axis=0)
    Pxx = np.delete(Pxx, np.s_[57:63 + 1], axis=0)
    Pxx = np.delete(Pxx, 0, axis=0)

    print("Cleaned but not standard")
    freqs = np.arange(Pxx.shape[0])
    plt.pcolormesh(freqs, bins, 10 * np.log10(np.transpose(Pxx)), cmap=plt.cm.jet)
    plt.colorbar()
    plt.ylabel('sec')
    plt.xlabel('Hz')
    plt.title('Spettrogramma')
    plt.show()
    plt.close()

    result = (10 * np.log10(np.transpose(Pxx)) - (10 * np.log10(np.transpose(Pxx))).min()) / (
            10 * np.log10(np.transpose(Pxx))).ptp()

    print("Standard")
    freqs = np.arange(result.shape[1])
    plt.pcolormesh(freqs, bins, result, cmap=plt.cm.jet)
    plt.colorbar()
    plt.ylabel('sec')
    plt.xlabel('Hz')
    plt.title('Spettrogramma')
    plt.show()
    plt.close()

    return result


class PreIntData:
    start = 0
    end = 0

    def __init__(self, s, e):
        self.start = s
        self.end = e


class FileData:
    start = 0
    end = 0
    nameFile = ""

    def __init__(self, s, e, nF):
        self.start = s
        self.end = e
        self.nameFile = nF


def createArrayIntervalData(fSummary):
    preictalInteval = []
    interictalInterval = []
    interictalInterval.append(PreIntData(datetime.min, datetime.max))
    files = []
    firstTime = True
    oldTime = datetime.min  # equivalente di 0 nelle date
    startTime = 0
    line = fSummary.readline()
    endS = datetime.min
    while (line):
        data = line.split(':')
        if (data[0] == "File Name"):
            nF = data[1].strip()
            s = getTime((fSummary.readline().split(": "))[1].strip())
            if (firstTime):
                interictalInterval[0].start = s
                firstTime = False
                startTime = s
            while s < oldTime:  # se cambia di giorno aggiungo 24 ore alla data
                s = s + timedelta(hours=24)
            oldTime = s
            endTimeFile = getTime((fSummary.readline().split(": "))[1].strip())
            while endTimeFile < oldTime:  # se cambia di giorno aggiungo 24 ore alla data
                endTimeFile = endTimeFile + timedelta(hours=24)
            oldTime = endTimeFile
            files.append(FileData(s, endTimeFile, nF))
            for j in range(0, int((fSummary.readline()).split(':')[1])):
                secSt = int(fSummary.readline().split(': ')[1].split(' ')[0])
                secEn = int(fSummary.readline().split(': ')[1].split(' ')[0])
                ss = s + timedelta(seconds=secSt) - timedelta(
                    minutes=_MINUTES_OF_DATA_BETWEEN_PRE_AND_SEIZURE + _MINUTES_OF_PREICTAL)
                if ((len(preictalInteval) == 0 or ss > endS) and ss - startTime > timedelta(minutes=20)):
                    ee = ss + timedelta(minutes=_MINUTES_OF_PREICTAL)
                    preictalInteval.append(PreIntData(ss, ee))
                endS = s + timedelta(seconds=secEn)
                ss = s + timedelta(seconds=secSt) - timedelta(hours=4)
                ee = s + timedelta(seconds=secEn) + timedelta(hours=4)
                if (interictalInterval[len(interictalInterval) - 1].start < ss and interictalInterval[
                    len(interictalInterval) - 1].end > ee):
                    interictalInterval[len(interictalInterval) - 1].end = ss
                    interictalInterval.append(PreIntData(ee, datetime.max))
                else:
                    if (interictalInterval[len(interictalInterval) - 1].start < ee):
                        interictalInterval[len(interictalInterval) - 1].start = ee
        line = fSummary.readline()
    fSummary.close()
    interictalInterval[len(interictalInterval) - 1].end = endTimeFile
    return preictalInteval, interictalInterval, files


def main():
    global SecondPartPathOutput
    global FirstPartPathOutput
    global legendOfOutput
    global nSpectogram
    global signalsBlock
    global isPreictal
    print("START \n")
    loadParametersFromFile("PARAMETERS_DATA_EDITING.txt")
    print("Parameters loaded")

    for indexPatient in range(0, len(patients)):
        print("Working on patient " + patients[indexPatient])
        legendOfOutput = ""
        allLegend = ""
        nSpectogram = 0

        SecondPartPathOutput = '/paz' + patients[indexPatient]
        f = loadSummaryPatient(indexPatient)
        preictalInfo, interictalInfo, filesInfo = createArrayIntervalData(f)
        if (patients[indexPatient] == "19"):
            preictalInfo.pop(0)  # Eliminazione dei dati della prima seizure perchè non viene considerata
        print("Summary patient loaded")

        print("START creation interictal spectrogram")
        totInst = 0
        # c=0
        # d=0
        interictalData = np.array([]).reshape(22, 0)
        indexInterictalSegment = 0
        isPreictal = ''
        for fInfo in filesInfo:
            fileS = fInfo.start
            fileE = fInfo.end
            intSegStart = interictalInfo[indexInterictalSegment].start
            intSegEnd = interictalInfo[indexInterictalSegment].end
            while (fileS > intSegEnd and indexInterictalSegment < len(interictalInfo)):
                indexInterictalSegment = indexInterictalSegment + 1
                intSegStart = interictalInfo[indexInterictalSegment].start
                intSegEnd = interictalInfo[indexInterictalSegment].end
            start = 0
            end = 0
            if (not fileE < intSegStart or fileS > intSegEnd):
                if (fileS >= intSegStart):
                    start = 0
                else:
                    start = (intSegStart - fileS).seconds
                if (fileE <= intSegEnd):
                    end = None
                else:
                    end = (intSegEnd - fileS).seconds
                tmpData = loadDataOfPatient(indexPatient, fInfo.nameFile)
                if (not end == None):
                    end = end * 256
                if (tmpData.shape[0] < 22):
                    print(patients[indexPatient] + "  HA UN NUMERO MINORE DI CANALI")
                else:
                    interictalData = np.concatenate((interictalData, tmpData[0:22, start * 256:end]), axis=1)
                    notUsed = createSpectrogram(interictalData)
                    totInst += interictalData.shape[1] / 256 - notUsed / 256
                    interictalData = np.delete(interictalData, np.s_[0:interictalData.shape[1] - notUsed], axis=1)

        # dimensione_finestra:lunghezza_dati_I=S:(lunghezza_dati_P-30_SEC_PER_OGNI_SEIZURE)
        S = (_SIZE_WINDOW_IN_SECONDS * (len(preictalInfo) * _MINUTES_OF_PREICTAL * 60 - _SIZE_WINDOW_IN_SECONDS * len(
            preictalInfo))) / totInst
        if (not (signalsBlock is None)):
            saveSignalsOnDisk(signalsBlock, nSpectogram)
        signalsBlock = None

        print("Spectrogram interictal: " + str(nSpectogram))
        print("Hours interictal: " + str(totInst / 60 / 60))
        legendOfOutput = str(nSpectogram) + "\n" + legendOfOutput
        legendOfOutput = "INTERICTAL" + "\n" + legendOfOutput
        legendOfOutput = "SEIZURE: " + str(len(preictalInfo)) + "\n" + legendOfOutput
        legendOfOutput = patients[indexPatient] + "\n" + legendOfOutput
        allLegend = legendOfOutput
        legendOfOutput = ''
        nSpectogram = 0
        print("END creation interictal spectrogram")
        # FINE ciclo gestione interictal data

        # INIZIO ciclo gestione preictal data
        print("START creation preictal spectrogram")
        isPreictal = 'P'
        contSeizure = -1
        for pInfo in preictalInfo:
            contSeizure = contSeizure + 1
            legendOfOutput = legendOfOutput + "SEIZURE " + str(contSeizure) + "\n"
            preictalData = np.array([]).reshape(22, 0)
            j = 0
            for j in range(0, len(filesInfo)):
                if (pInfo.start >= filesInfo[j].start and pInfo.start < filesInfo[j].end):
                    break
            start = (pInfo.start - filesInfo[j].start).seconds
            if (start < 0):
                start = 0
            end = None
            tmpData = []
            if (pInfo.end <= filesInfo[j].end):
                end = (pInfo.end - filesInfo[j].start).seconds
                tmpData = loadDataOfPatient(indexPatient, filesInfo[j].nameFile)
                preictalData = np.concatenate((preictalData, tmpData[0:22, start * 256:end * 256]), axis=1)
            else:
                tmpData = loadDataOfPatient(indexPatient, filesInfo[j].nameFile)
                preictalData = np.concatenate((preictalData, tmpData[0:22, start * 256:]), axis=1)
                end = (pInfo.end - filesInfo[j + 1].start).seconds
                tmpData = loadDataOfPatient(indexPatient, filesInfo[j + 1].nameFile)
                preictalData = np.concatenate((preictalData, tmpData[0:22, 0:end * 256]), axis=1)
            notUsed = createSpectrogram(preictalData, S=S)
            if (not (signalsBlock is None)):
                saveSignalsOnDisk(signalsBlock, nSpectogram)
            signalsBlock = None

        allLegend = allLegend + "\n" + "PREICTAL" + "\n" + str(nSpectogram) + "\n" + legendOfOutput
        print("Spectrogram preictal: " + str(nSpectogram))
        print("SEIZURE: " + str(len(preictalInfo)))
        print("END creation preictal spectrogram")
        # FINE ciclo gestione preictal data'''

        print("START creation \'real\' preictal spectrogram")
        isPreictal = 'P_R'
        nSpectogram = 0
        contSeizure = -1
        S = 0
        legendOfOutput = ''
        for pInfo in preictalInfo:
            contSeizure = contSeizure + 1
            legendOfOutput = legendOfOutput + "SEIZURE " + str(contSeizure) + "\n"
            preictalData = np.array([]).reshape(22, 0)
            j = 0
            for j in range(0, len(filesInfo)):
                if (pInfo.start >= filesInfo[j].start and pInfo.start < filesInfo[j].end):
                    break
            start = (pInfo.start - filesInfo[j].start).seconds
            if (start < 0):
                start = 0
            end = None
            tmpData = []
            if (pInfo.end <= filesInfo[j].end):
                end = (pInfo.end - filesInfo[j].start).seconds
                tmpData = loadDataOfPatient(indexPatient, filesInfo[j].nameFile)
                preictalData = np.concatenate((preictalData, tmpData[0:22, start * 256:end * 256]), axis=1)
            else:
                tmpData = loadDataOfPatient(indexPatient, filesInfo[j].nameFile)
                preictalData = np.concatenate((preictalData, tmpData[0:22, start * 256:]), axis=1)
                end = (pInfo.end - filesInfo[j + 1].start).seconds
                tmpData = loadDataOfPatient(indexPatient, filesInfo[j + 1].nameFile)
                preictalData = np.concatenate((preictalData, tmpData[0:22, 0:end * 256]), axis=1)
            notUsed = createSpectrogram(preictalData, S=S)
            if (not (signalsBlock is None)):
                saveSignalsOnDisk(signalsBlock, nSpectogram)
            signalsBlock = None

        allLegend = allLegend + "\n" + "REAL_PREICTAL" + "\n" + str(nSpectogram) + "\n" + legendOfOutput
        print("Spectrogram \'REAL\' preictal: " + str(nSpectogram))
        print("END creation \'real\' preictal spectrogram")

        text_file = open(FirstPartPathOutput + SecondPartPathOutput + "/legendAllData.txt", "w")
        text_file.write(allLegend)
        text_file.close()
        print("Legend saved on disk")
        print('\n')
    print("END")


if __name__ == '__main__':
    main()


In [0]:
import os
# os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"   # see issue #152
# os.environ["CUDA_VISIBLE_DEVICES"] = ""
# FINE codice per allenare la rete sulla cpu

import keras
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Conv3D, Dropout, Flatten, BatchNormalization
from keras.callbacks import EarlyStopping
from random import shuffle
import math

# to plot the model
# from keras.utils.vis_utils import plot_model
# from keras.models import load_model
# Returns a compiled model identical to the saved one
# model = load_model('my_model.h5')

PathSpectogramFolder = './project/dataset/spectograms'
OutputPath = './project/dataset/results.txt'
OutputPathModels = './project/dataset/'
interictalSpectograms = []
preictalSpectograms = []
preictalRealSpectograms = []
#patients = ["01", "02", "03", "05", "09", "10", "13", "14", "18", "19", "20", "21", "23"]
#patients = ["09", "10", "14", "18"]
nSeizure = 0


def loadParametersFromFile(filePath):
    global PathSpectogramFolder
    global OutputPath
    global OutputPathModels
    if (os.path.isfile(filePath)):
        with open(filePath, "r") as f:
            line = f.readline()
            if (line.split(":")[0] == "PathSpectogramFolder"):
                PathSpectogramFolder = line.split(":")[1].strip()
            line = f.readline()
            if (line.split(":")[0] == "OutputPath"):
                OutputPath = line.split(":")[1].strip()
            line = f.readline()
            if (line.split(":")[0] == "OutputPathModels"):
                OutputPathModels = line.split(":")[1].strip()


def loadSpectogramData(indexPat):
    global interictalSpectograms
    global preictalSpectograms
    global preictalRealSpectograms
    global nSeizure
    nFileForSeizure = 0

    interictalSpectograms = []
    preictalSpectograms = []
    preictalRealSpectograms = []

    f = open(PathSpectogramFolder + '/paz' + patients[indexPat] + '/legendAllData.txt', 'r')
    line = f.readline()
    while (not "SEIZURE" in line):
        line = f.readline()
    nSeizure = int(line.split(":")[1].strip())
    line = f.readline()
    line = f.readline()  # legge il numero di spectogrammi. non lo salvo dato che non mi serve
    nSpectograms = int(line.strip())
    nFileForSeizure = math.ceil(math.ceil(nSpectograms / 50) / nSeizure)
    line = f.readline()  # leggo il percorso del primo file

    # Lettura path files Interictal
    cont = -1
    indFilePathRead = 0
    while ("npy" in line and indFilePathRead < nSeizure * nFileForSeizure):
        if (indFilePathRead % nFileForSeizure == 0):
            interictalSpectograms.append([])
            cont = cont + 1
            interictalSpectograms[cont].append(line.split(' ')[2].rstrip())  # .rstrip() remove \n
            indFilePathRead = indFilePathRead + 1
        else:
            if (len(line.split(' ')) >= 3):
                interictalSpectograms[cont].append(line.split(' ')[2].rstrip())
            indFilePathRead = indFilePathRead + 1

        line = f.readline()
    line = f.readline()  # leggo PREICTAL
    line = f.readline()  # leggo n° spectogram
    line = f.readline()  # leggo n°seizure(SEIZURE X)

    cont = -1
    indFilePathRead = 0
    # while(line and indFilePathRead<nSeizure*nFileForSeizure):
    while (line.strip() != ""):
        if ("SEIZURE" in line):
            line = f.readline()  # ho letto n°seizure(SEIZURE X) perciò scorro in avanti
            if (len(line.split(' ')) >= 3):
                preictalSpectograms.append([])
                cont = cont + 1
                preictalSpectograms[cont].append(line.split(' ')[2].rstrip())
                indFilePathRead = indFilePathRead + 1
        else:
            if (len(line.split(' ')) >= 3):
                preictalSpectograms[cont].append(line.split(' ')[2].rstrip())
            indFilePathRead = indFilePathRead + 1

        line = f.readline()

    line = f.readline()  # leggo REAL_PREICTAL
    line = f.readline()  # leggo n° spectogram
    line = f.readline()  # leggo n°seizure(SEIZURE X)

    cont = -1
    while (line):
        if ("SEIZURE" in line):
            line = f.readline()
            preictalRealSpectograms.append([])
            cont = cont + 1
            preictalRealSpectograms[cont].append(line.split(' ')[2].rstrip())
        else:
            preictalRealSpectograms[cont].append(line.split(' ')[2].rstrip())

        line = f.readline()
    f.close()


def createModel():
    input_shape = (1, 22, 59, 114)
    model = Sequential()
    # C1
    model.add(
        Conv3D(16, (22, 5, 5), strides=(1, 2, 2), padding='valid', activation='relu', data_format="channels_first",
               input_shape=input_shape))
    model.add(keras.layers.MaxPooling3D(pool_size=(1, 2, 2), data_format="channels_first", padding='same'))
    model.add(BatchNormalization())

    # C2
    model.add(Conv3D(32, (1, 3, 3), strides=(1, 1, 1), padding='valid', data_format="channels_first",
                     activation='relu'))  # incertezza se togliere padding
    model.add(keras.layers.MaxPooling3D(pool_size=(1, 2, 2), data_format="channels_first", ))
    model.add(BatchNormalization())

    # C3
    model.add(Conv3D(64, (1, 3, 3), strides=(1, 1, 1), padding='valid', data_format="channels_first",
                     activation='relu'))  # incertezza se togliere padding
    model.add(keras.layers.MaxPooling3D(pool_size=(1, 2, 2), data_format="channels_first", ))
    model.add(BatchNormalization())

    model.add(Flatten())
    model.add(Dropout(0.5))
    model.add(Dense(256, activation='sigmoid'))
    model.add(Dropout(0.5))
    model.add(Dense(2, activation='softmax'))

    opt_adam = keras.optimizers.Adam(lr=0.00001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
    model.compile(loss='categorical_crossentropy', optimizer=opt_adam, metrics=['accuracy'])

    return model


def getFilesPathWithoutSeizure(indexSeizure, indexPat):
    filesPath = []
    for i in range(0, nSeizure):
        if (i != indexSeizure):
            filesPath.extend(interictalSpectograms[i])
            filesPath.extend(preictalSpectograms[i])
    shuffle(filesPath)
    return filesPath


def generate_arrays_for_training(indexPat, paths, start=0, end=100):
    while True:
        from_ = int(len(paths) / 100 * start)
        to_ = int(len(paths) / 100 * end)
        for i in range(from_, int(to_)):
            f = paths[i]
            x = np.load(PathSpectogramFolder + f)
            x = np.array([x])
            x = x.swapaxes(0, 1)
            if ('P' in f):
                y = np.repeat([[0, 1]], x.shape[0], axis=0)
            else:
                y = np.repeat([[1, 0]], x.shape[0], axis=0)
            yield (x, y)


def generate_arrays_for_predict(indexPat, paths, start=0, end=100):
    while True:
        from_ = int(len(paths) / 100 * start)
        to_ = int(len(paths) / 100 * end)
        for i in range(from_, int(to_)):
            f = paths[i]
            x = np.load(PathSpectogramFolder + f)
            x = np.array([x])
            x = x.swapaxes(0, 1)
            yield (x)


class EarlyStoppingByLossVal(keras.callbacks.Callback):
    def __init__(self, monitor='val_loss', value=0.00001, verbose=0, lower=True):
        super(keras.callbacks.Callback, self).__init__()
        self.monitor = monitor
        self.value = value
        self.verbose = verbose
        self.lower = lower

    def on_epoch_end(self, epoch, logs={}):
        current = logs.get(self.monitor)
        if self.lower:
            if current < self.value:
                if self.verbose > 0:
                    print("Epoch %05d: early stopping THR" % epoch)
                self.model.stop_training = True
        else:
            if current > self.value:
                if self.verbose > 0:
                    print("Epoch %05d: early stopping THR" % epoch)
                self.model.stop_training = True


def main():
    print("START")
    if not os.path.exists(OutputPathModels):
        os.makedirs(OutputPathModels)
    loadParametersFromFile("PARAMETERS_CNN.txt")
    # callback=EarlyStopping(monitor='val_acc', min_delta=0, patience=0, verbose=0, mode='auto', baseline=None)
    callback = EarlyStoppingByLossVal(monitor='val_acc', value=0.975, verbose=1, lower=False)
    print("Parameters loaded")

    for indexPat in range(0, len(patients)):
        print('Patient ' + patients[indexPat])
        if not os.path.exists(OutputPathModels + "ModelPat" + patients[indexPat] + "/"):
            os.makedirs(OutputPathModels + "ModelPat" + patients[indexPat] + "/")
        loadSpectogramData(indexPat)
        print('Spectograms data loaded')

        result = 'Patient ' + patients[indexPat] + '\n'
        result = 'Out Seizure, True Positive, False Positive, False negative, Second of Inter in Test, Sensitivity, FPR \n'
        for i in range(0, nSeizure):
            print('SEIZURE OUT: ' + str(i + 1))

            print('Training start')
            model = createModel()
            filesPath = getFilesPathWithoutSeizure(i, indexPat)

            model.fit_generator(generate_arrays_for_training(indexPat, filesPath, end=75),
                                # end=75),#It take the first 75%
                                validation_data=generate_arrays_for_training(indexPat, filesPath, start=75),
                                # start=75), #It take the last 25%
                                # steps_per_epoch=10000, epochs=10)
                                steps_per_epoch=int((len(filesPath) - int(len(filesPath) / 100 * 25))),  # *25),
                                validation_steps=int((len(filesPath) - int(len(filesPath) / 100 * 75))),  # *75),
                                verbose=2,
                                epochs=300, max_queue_size=2, shuffle=True, callbacks=[
                    callback])  # 100 epochs è meglio #aggiungere criterio di stop in base accuratezza
            print('Training end')

            print('Testing start')
            filesPath = interictalSpectograms[i]
            interPrediction = model.predict_generator(generate_arrays_for_predict(indexPat, filesPath),
                                                      max_queue_size=4, steps=len(filesPath))
            filesPath = preictalRealSpectograms[i]
            preictPrediction = model.predict_generator(generate_arrays_for_predict(indexPat, filesPath),
                                                       max_queue_size=4, steps=len(filesPath))
            print('Testing end')

            # Creates a HDF5 file 
            model.save(
                OutputPathModels + "ModelPat" + patients[indexPat] + "/" + 'ModelOutSeizure' + str(i + 1) + '.h5')
            print("Model saved")

            # to plot the model
            # plot_model(model, to_file="CNNModel", show_shapes=True, show_layer_names=True)

            if not os.path.exists(OutputPathModels + "OutputTest" + "/"):
                os.makedirs(OutputPathModels + "OutputTest" + "/")
            np.savetxt(OutputPathModels + "OutputTest" + "/" + "Int_" + patients[indexPat] + "_" + str(i + 1) + ".csv",
                       interPrediction, delimiter=",")
            np.savetxt(OutputPathModels + "OutputTest" + "/" + "Pre_" + patients[indexPat] + "_" + str(i + 1) + ".csv",
                       preictPrediction, delimiter=",")

            secondsInterictalInTest = len(
                interictalSpectograms[i]) * 50 * 30  # 50 spectograms for file, 30 seconds for each spectogram
            acc = 0  # accumulator
            fp = 0
            tp = 0
            fn = 0
            lastTenResult = list()

            for el in interPrediction:
                if (el[1] > 0.5):
                    acc = acc + 1
                    lastTenResult.append(1)
                else:
                    lastTenResult.append(0)
                if (len(lastTenResult) > 10):
                    acc = acc - lastTenResult.pop(0)
                if (acc >= 8):
                    fp = fp + 1
                    lastTenResult = list()
                    acc = 0

            lastTenResult = list()
            for el in preictPrediction:
                if (el[1] > 0.5):
                    acc = acc + 1
                    lastTenResult.append(1)
                else:
                    lastTenResult.append(0)
                if (len(lastTenResult) > 10):
                    acc = acc - lastTenResult.pop(0)
                if (acc >= 8):
                    tp = tp + 1
                else:
                    if (len(lastTenResult) == 10):
                        fn = fn + 1

            sensitivity = tp / (tp + fn)
            FPR = fp / (secondsInterictalInTest / (60 * 60))

            result = result + str(i + 1) + ',' + str(tp) + ',' + str(fp) + ',' + str(fn) + ',' + str(
                secondsInterictalInTest) + ','
            result = result + str(sensitivity) + ',' + str(FPR) + '\n'
            print('True Positive, False Positive, False negative, Second of Inter in Test, Sensitivity, FPR')
            print(str(tp) + ',' + str(fp) + ',' + str(fn) + ',' + str(secondsInterictalInTest) + ',' + str(
                sensitivity) + ',' + str(FPR))
        with open(OutputPath, "a+") as myfile:
            myfile.write(result)


if __name__ == '__main__':
    main()


In [0]:
!zip -r /content/file.zip /content/project


In [0]:
from google.colab import files
files.download("/content/file.zip")

----------------------------------------
Exception happened during processing of request from ('::ffff:127.0.0.1', 36150, 0, 0)
Traceback (most recent call last):
  File "/usr/lib/python3.6/socketserver.py", line 317, in _handle_request_noblock
    self.process_request(request, client_address)
  File "/usr/lib/python3.6/socketserver.py", line 348, in process_request
    self.finish_request(request, client_address)
  File "/usr/lib/python3.6/socketserver.py", line 361, in finish_request
    self.RequestHandlerClass(request, client_address, self)
  File "/usr/lib/python3.6/socketserver.py", line 721, in __init__
    self.handle()
  File "/usr/lib/python3.6/http/server.py", line 418, in handle
    self.handle_one_request()
  File "/usr/lib/python3.6/http/server.py", line 406, in handle_one_request
    method()
  File "/usr/lib/python3.6/http/server.py", line 639, in do_GET
    self.copyfile(f, self.wfile)
  File "/usr/lib/python3.6/http/server.py", line 800, in copyfile
    shutil.copyfil

In [0]:
  from keras.models import load_model as model

modeld = model("/content/project/dataset/ModelPat01/ModelOutSeizure1.h5")

In [0]:
import os
# os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"   # see issue #152
# os.environ["CUDA_VISIBLE_DEVICES"] = ""
# FINE codice per allenare la rete sulla cpu

import keras
import numpy as np
from keras.models import load_model as model
from keras.models import Sequential
from keras.layers import Dense, Conv3D, Dropout, Flatten, BatchNormalization
from keras.callbacks import EarlyStopping
from random import shuffle
import math

# to plot the model
# from keras.utils.vis_utils import plot_model
# from keras.models import load_model
# Returns a compiled model identical to the saved one
# model = load_model('my_model.h5')

PathSpectogramFolder = './project/dataset/spectograms'
OutputPath = './project/dataset/results.txt'
OutputPathModels = './project/dataset/'
interictalSpectograms = []
preictalSpectograms = []
preictalRealSpectograms = []
#patients = ["01", "02", "03", "05", "09", "10", "13", "14", "18", "19", "20", "21", "23"]
patients = ["02"]
nSeizure = 0


def loadParametersFromFile(filePath):
    global PathSpectogramFolder
    global OutputPath
    global OutputPathModels
    if (os.path.isfile(filePath)):
        with open(filePath, "r") as f:
            line = f.readline()
            if (line.split(":")[0] == "PathSpectogramFolder"):
                PathSpectogramFolder = line.split(":")[1].strip()
            line = f.readline()
            if (line.split(":")[0] == "OutputPath"):
                OutputPath = line.split(":")[1].strip()
            line = f.readline()
            if (line.split(":")[0] == "OutputPathModels"):
                OutputPathModels = line.split(":")[1].strip()


def loadSpectogramData(indexPat):
    global interictalSpectograms
    global preictalSpectograms
    global preictalRealSpectograms
    global nSeizure
    nFileForSeizure = 0

    interictalSpectograms = []
    preictalSpectograms = []
    preictalRealSpectograms = []

    f = open(PathSpectogramFolder + '/paz' + patients[indexPat] + '/legendAllData.txt', 'r')
    line = f.readline()
    while (not "SEIZURE" in line):
        line = f.readline()
    nSeizure = int(line.split(":")[1].strip())
    line = f.readline()
    line = f.readline()  # legge il numero di spectogrammi. non lo salvo dato che non mi serve
    nSpectograms = int(line.strip())
    nFileForSeizure = math.ceil(math.ceil(nSpectograms / 50) / nSeizure)
    line = f.readline()  # leggo il percorso del primo file

    # Lettura path files Interictal
    cont = -1
    indFilePathRead = 0
    while ("npy" in line and indFilePathRead < nSeizure * nFileForSeizure):
        if (indFilePathRead % nFileForSeizure == 0):
            interictalSpectograms.append([])
            cont = cont + 1
            interictalSpectograms[cont].append(line.split(' ')[2].rstrip())  # .rstrip() remove \n
            indFilePathRead = indFilePathRead + 1
        else:
            if (len(line.split(' ')) >= 3):
                interictalSpectograms[cont].append(line.split(' ')[2].rstrip())
            indFilePathRead = indFilePathRead + 1

        line = f.readline()
    line = f.readline()  # leggo PREICTAL
    line = f.readline()  # leggo n° spectogram
    line = f.readline()  # leggo n°seizure(SEIZURE X)

    cont = -1
    indFilePathRead = 0
    # while(line and indFilePathRead<nSeizure*nFileForSeizure):
    while (line.strip() != ""):
        if ("SEIZURE" in line):
            line = f.readline()  # ho letto n°seizure(SEIZURE X) perciò scorro in avanti
            if (len(line.split(' ')) >= 3):
                preictalSpectograms.append([])
                cont = cont + 1
                preictalSpectograms[cont].append(line.split(' ')[2].rstrip())
                indFilePathRead = indFilePathRead + 1
        else:
            if (len(line.split(' ')) >= 3):
                preictalSpectograms[cont].append(line.split(' ')[2].rstrip())
            indFilePathRead = indFilePathRead + 1

        line = f.readline()

    line = f.readline()  # leggo REAL_PREICTAL
    line = f.readline()  # leggo n° spectogram
    line = f.readline()  # leggo n°seizure(SEIZURE X)

    cont = -1
    while (line):
        if ("SEIZURE" in line):
            line = f.readline()
            preictalRealSpectograms.append([])
            cont = cont + 1
            preictalRealSpectograms[cont].append(line.split(' ')[2].rstrip())
        else:
            preictalRealSpectograms[cont].append(line.split(' ')[2].rstrip())

        line = f.readline()
    f.close()



def getFilesPathWithoutSeizure(indexSeizure, indexPat):
    filesPath = []
    for i in range(0, nSeizure):
        if (i != indexSeizure):
            filesPath.extend(interictalSpectograms[i])
            filesPath.extend(preictalSpectograms[i])
    shuffle(filesPath)
    return filesPath


def generate_arrays_for_training(indexPat, paths, start=0, end=100):
    while True:
        from_ = int(len(paths) / 100 * start)
        to_ = int(len(paths) / 100 * end)
        for i in range(from_, int(to_)):
            f = paths[i]
            x = np.load(PathSpectogramFolder + f)
            x = np.array([x])
            x = x.swapaxes(0, 1)
            if ('P' in f):
                y = np.repeat([[0, 1]], x.shape[0], axis=0)
            else:
                y = np.repeat([[1, 0]], x.shape[0], axis=0)
            yield (x, y)


def generate_arrays_for_predict(indexPat, paths, start=0, end=100):
    while True:
        from_ = int(len(paths) / 100 * start)
        to_ = int(len(paths) / 100 * end)
        for i in range(from_, int(to_)):
            f = paths[i]
            x = np.load(PathSpectogramFolder + f)
            x = np.array([x])
            x = x.swapaxes(0, 1)
            yield (x)


class EarlyStoppingByLossVal(keras.callbacks.Callback):
    def __init__(self, monitor='val_loss', value=0.00001, verbose=0, lower=True):
        super(keras.callbacks.Callback, self).__init__()
        self.monitor = monitor
        self.value = value
        self.verbose = verbose
        self.lower = lower

    def on_epoch_end(self, epoch, logs={}):
        current = logs.get(self.monitor)
        if self.lower:
            if current < self.value:
                if self.verbose > 0:
                    print("Epoch %05d: early stopping THR" % epoch)
                self.model.stop_training = True
        else:
            if current > self.value:
                if self.verbose > 0:
                    print("Epoch %05d: early stopping THR" % epoch)
                self.model.stop_training = True


def main():
    print("START")
    if not os.path.exists(OutputPathModels):
        os.makedirs(OutputPathModels)
    loadParametersFromFile("PARAMETERS_CNN.txt")
    # callback=EarlyStopping(monitor='val_acc', min_delta=0, patience=0, verbose=0, mode='auto', baseline=None)
    callback = EarlyStoppingByLossVal(monitor='val_acc', value=0.975, verbose=1, lower=False)
    print("Parameters loaded")

    for indexPat in range(0, len(patients)):
        print('Patient ' + patients[indexPat])
        if not os.path.exists(OutputPathModels + "ModelPat" + patients[indexPat] + "/"):
            os.makedirs(OutputPathModels + "ModelPat" + patients[indexPat] + "/")
        loadSpectogramData(indexPat)
        print('Spectograms data loaded')

        result = 'Patient ' + patients[indexPat] + '\n'
        result = 'Out Seizure, True Positive, False Positive, False negative, Second of Inter in Test, Sensitivity, FPR \n'
        for i in range(0, nSeizure):
            print('SEIZURE OUT: ' + str(i + 1))
          
            
            modeld=model(OutputPathModels + "ModelPat" + "01"+ "/" + 'ModelOutSeizure' + str(i + 1) + '.h5')
            filesPath = interictalSpectograms[i]
            interPrediction = modeld.predict_generator(generate_arrays_for_predict(indexPat, filesPath),
                                                      max_queue_size=4, steps=len(filesPath))
            filesPath = preictalRealSpectograms[i]
            preictPrediction = modeld.predict_generator(generate_arrays_for_predict(indexPat, filesPath),
                                                       max_queue_size=4, steps=len(filesPath))
            print("prediction:",preictPrediction)
           
            
            secondsInterictalInTest = len(
                interictalSpectograms[i]) * 50 * 30  # 50 spectograms for file, 30 seconds for each spectogram
            acc = 0  # accumulator
            fp = 0
            tp = 0
            fn = 0
            lastTenResult = list()

            for el in interPrediction:
                if (el[1] > 0.5):
                    acc = acc + 1
                    lastTenResult.append(1)
                else:
                    lastTenResult.append(0)
                if (len(lastTenResult) > 10):
                    acc = acc - lastTenResult.pop(0)
                if (acc >= 8):
                    fp = fp + 1
                    lastTenResult = list()
                    acc = 0

            lastTenResult = list()
            for el in preictPrediction:
                if (el[1] > 0.5):
                    acc = acc + 1
                    lastTenResult.append(1)
                else:
                    lastTenResult.append(0)
                if (len(lastTenResult) > 10):
                    acc = acc - lastTenResult.pop(0)
                if (acc >= 8):
                    tp = tp + 1
                else:
                    if (len(lastTenResult) == 10):
                        fn = fn + 1

            sensitivity = tp / (tp + fn)
            FPR = fp / (secondsInterictalInTest / (60 * 60))

            result = result + str(i + 1) + ',' + str(tp) + ',' + str(fp) + ',' + str(fn) + ',' + str(
                secondsInterictalInTest) + ','
            result = result + str(sensitivity) + ',' + str(FPR) + '\n'
            print('True Positive, False Positive, False negative, Second of Inter in Test, Sensitivity, FPR')
            print(str(tp) + ',' + str(fp) + ',' + str(fn) + ',' + str(secondsInterictalInTest) + ',' + str(
                sensitivity) + ',' + str(FPR))
        with open(OutputPath, "a+") as myfile:
            myfile.write(result)
            print(result)

if __name__ == '__main__':
    main()


In [0]:
predict = modeld.predict(generate_arrays_for_predict(indexPat, filesPath))
predict